# Exercises day 03

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from typing import Callable

def compute_confidence_interval_sample(data : list[float], confidence : float = 0.95) -> list[float]:
    """
        Compute the confidence interval of the mean of the data.
        :param data: list of data
        :param confidence: confidence level
        
        :return: confidence interval of the mean of the data
    """
    N = len(data)
    alpha = 1 - confidence

    data = np.array(data)
    #conf_mean = [data.mean - confidence * data.std()/np.sqrt(N), data.mean + confidence * data.std()/np.sqrt(N)]
    conf_mean = [data.mean() + data.std()/np.sqrt(N) * scipy.stats.t.cdf(alpha/2, N - 1), data.mean() + data.std()/np.sqrt(N) * scipy.stats.t.cdf(1-alpha/2, N - 1)]
    return conf_mean
    

## Ex01
The event for the blocking system is has the possible following event:

* Arrival of a customer 
* Customer is served
* Customer is blocked, as all $m$ service unints is occupied.

For part 1 the arrival process is modelled as a Poisson process, and the service time distribution as exponential.

In [2]:
# Define blocking system
m = 10              # service units
SERVICE_MEAN = 8    # 1/mean service time
ARRIVAL_MEAN = 1    # mean arrival time      


Class for service units

In [3]:
class Event:
    def __init__(self, event_type : str, time : float, ) -> None:
        if event_type.lower() not in ['arrival', 'departure']:
            raise ValueError('event_type must be either arrival or departure')
        self.event_type = event_type
        self.time = time
       

In [4]:
def sample_arrival_poisson_process() -> float:
    """
        Sample the arrival time from a Poisson process.
        
        :return: arrival time (as a float)
    """
    return np.random.exponential(ARRIVAL_MEAN) # TODO: check if is it not 1/arrial_mean


def sample_service_time_exponential() -> float:
    """
        Sample the service time from an exponential distribution.
        
        :return: service time (as a float)
    """
    return np.random.exponential(SERVICE_MEAN) 


def check_available_service(service_units : list[bool]) -> tuple[int, bool]:
    """
    
    """
    for indx, unit_occupied in enumerate(service_units):
        if not unit_occupied:
            return indx, True
        
    return None, False


def apend_event(event_list : list[Event], event_to_append : Event) -> list[Event]:
    """

    """
    for indx, event in enumerate(event_list):
        
        if event.time > event_to_append.time:
            event_list.insert(indx, event_to_append)
            return event_list
        
    event_list.append(event_to_append)
    return event_list

Simulation:

In [5]:
def blocking_simulation(
    simulation_runs : int = 10,
    m : int = 10,
    N : int = 10000,
    sample_arrival : Callable[[], float] = sample_arrival_poisson_process,
    sample_service_time : Callable[[], float] = sample_service_time_exponential
    ) -> list[float]:
    """
        A function for runinng multiple simulations of a blocking system.
        
        :param simulation_runs: number of simulations to run
        :param m: number of service units
        :param N: number of customers
        :param sample_arrival: function for sampling arrival time
        :param sample_service_time: function for sampling service time
        
        :return: list of blocked fractions
    """
    # NOTE: Maybe use burn in period...?
    
    blocked_fractions = []
    for i in range(simulation_runs):
        print(f"run {i+1}")
        custmer_count = 0
        global_time = 0
        event_counter = 0
        block_count = 0

        # lists
        event_list = []
        service_units_status = [False for _ in range(m)]    # <-- Indicates whether the service units are occupied or not
        
        # First arrival
        first_arrival = sample_arrival()
        event_list.append(Event('arrival', global_time + first_arrival))
        global_time += first_arrival
        event_list.append(Event('departure', global_time + sample_service_time()))
        service_units_status[0] = True # <-- unit 1 is occupied

        while custmer_count < N:
            
            current_event = event_list[event_counter]

            # Increment global time
            global_time = current_event.time

            if current_event.event_type == 'arrival':
                custmer_count += 1

                # Check for free service units
                indx, available = check_available_service(service_units_status)
                
                if available:
                    # Insert departure event and depend to eventlist
                    
            
                    departure_event = Event('departure', global_time + sample_service_time())
                    event_list = apend_event(event_list, departure_event)

                    # Take service unit
                    service_units_status[indx] = True # <-- unit indx is occupied

                if not available:
                    # Costumer blocked
                    block_count += 1
                
                # insert time for next arrival
                arrival_event = Event('arrival', global_time + sample_arrival())
                event_list = apend_event(event_list, arrival_event)

            elif current_event.event_type == 'departure': 
                # Free the service unit for the current departure event
                for indx, unit_occupied in enumerate(service_units_status):
                    if unit_occupied:
                        service_units_status[indx] = False # <-- unit indx is free
                        break
                        
            # increment event counter
            event_counter += 1
        
        blocked_fractions.append(block_count / N)
    
    return blocked_fractions

In [6]:
block_fraction = blocking_simulation()

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")


run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
[0.1226, 0.1243, 0.1201, 0.121, 0.1295, 0.1321, 0.1226, 0.1152, 0.1346, 0.1189]
Confidence interval for the mean blocking fraction: [0.1250295758796117, 0.12560615549668985]


Exact solution

In [7]:
A = ARRIVAL_MEAN * SERVICE_MEAN

B = (A**m/np.math.factorial(m))/np.sum([(A**i)/np.math.factorial(i) for i in range(m+1)])
print(f"Exact blocking probability: {B}")

Exact blocking probability: 0.12166106425295149


The simulation is sufficiently close to the found exact solution.

## EX02
The model is now redefined for more other arrival processes of the costumers.

### a) Erlang distributed arrival


In [8]:
def sample_arrival_erlang() -> float:
    """
        Sample the arrival time from an erlang distribution with a = 1.
        (NOTE: a = 1, is apparently the mean value in SciPys implementation, idk we asked a TA)
        
        :return: arrival time (as a float)
    """
    return scipy.stats.erlang.rvs(a = 1)

In [9]:
block_fraction = blocking_simulation(sample_arrival=sample_arrival_erlang)

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")

run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
[0.1285, 0.1289, 0.1149, 0.1201, 0.1094, 0.1183, 0.1242, 0.1204, 0.1263, 0.1209]
Confidence interval for the mean blocking fraction: [0.12212550248251536, 0.12269958242097362]


### b) hyper exponential distribution
For sampling from the hyper exponential distribution we utilize the Composition methods from last week

In [10]:
HYPER_PROB = 0.8
HYPER_MEAN = [1/0.8333, 1/5.]

def sample_arrival_hyper_exponential() -> float:
    """
    """
    p = np.random.binomial(n=1,p= HYPER_PROB)

    if p:
        return np.random.exponential(HYPER_MEAN[0])
    else:
        return np.random.exponential(HYPER_MEAN[1])


Simulation:

In [11]:
block_fraction = blocking_simulation(sample_arrival=sample_arrival_hyper_exponential)

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")

run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
[0.1445, 0.1413, 0.1349, 0.1347, 0.1324, 0.1393, 0.1412, 0.1313, 0.1393, 0.1394]
Confidence interval for the mean blocking fraction: [0.13848508018972264, 0.13888707633834863]


## Ex03

**We reuse Poisson from** *Ex01* **but then we use some new distributions for sampling the service time**
* **Constant service time**
* **Paretto distributed with $k = 1.05$ or $k = 2.05$**
* **Choose some other distributions of our own volition**

### a) Constant service time

In [12]:
sample_service_time_constant = lambda : 8

In [13]:
block_fraction = blocking_simulation(sample_service_time=sample_service_time_constant)

block_fraction_conf = compute_confidence_interval_sample(block_fraction)

print(f"The fraction of blocked customers \n{block_fraction}")
print(f"Confidence interval for the mean blocking fraction: {block_fraction_conf}")

run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
The fraction of blocked customers 
[0.1208, 0.1266, 0.1284, 0.1288, 0.1243, 0.1182, 0.1229, 0.1219, 0.1227, 0.1213]
Confidence interval for the mean blocking fraction: [0.12411488928767918, 0.1244369925590228]


### b) Pareto distributed


In [14]:
k = 1.05 

def sample_service_time_pareto() -> float:
    """
        Sample the service time from an exponential distribution.
        
        :return: service time (as a float)
    """
    U = np.random.uniform(0.0, 1.0, 1)
    beta = (k - 1) / k
    return beta * (U**(-1 / k) - 1)


def pareto_samples(k, num_samples: int):
    U = np.random.uniform(0.0, 1.0, num_samples)
    beta = 8*(k - 1) / k
    X = beta*(U**(-1/k))

    return X

In [15]:
k_list = np.linspace(1.05, 2.05, 2) # <-- might just be one of the most insane lines of code I have ever seen
k_list = np.array([1.05, 2.05])

def sample_service_time_pareto(k) -> float:
    """
        Sample the service time from an pareto distribution.
        
        :return: service time (as a float) will be above 1
    """
    return scipy.stats.pareto.rvs(k)


Simulation with the different k values

In [16]:
pareto_confs = []
for k in k_list:
    print(f"Pareto with k={k}")
    temp_lambda = lambda : sample_service_time_pareto(k)
    block_fraction = blocking_simulation(sample_service_time=temp_lambda)
    block_fraction_conf = compute_confidence_interval_sample(block_fraction)

    pareto_confs.append(block_fraction_conf)    
    
print(pareto_confs)

Pareto with k=1.05
run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
Pareto with k=2.05
run 1
run 2
run 3
run 4
run 5
run 6
run 7
run 8
run 9
run 10
[[0.12144097224093167, 0.1232027714859298], [1.483543639917555e-05, 1.7802747638151312e-05]]


Does not work. 

In [17]:
k = 1.05
U = np.random.uniform(0.0, 1.0, 1000000)
beta = 8*((k - 1) / k)
samples = beta * (np.power(U, (-1 / k)) - 1)
#samples = beta * (U**(-1 / k) - 1)

print(samples.mean())

3.549655405753638
